# IAU 2023/2024
## **Autori:** Laura Fulajtárová (50%), Bruno Kristián (50%)

# Fáza 3 - Strojové učenie 

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import copy

from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.tree import plot_tree
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import MinMaxScaler, PowerTransformer
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import VotingClassifier

import matplotlib.pyplot as plt

from collections import Counter

from IPython.display import display

### OneR algoritmus

Dočasne sme spojili dáta z trénovacej a testovacej množiny, aby sme mohli vytvoriť OneR model. 

In [ ]:
X_train = pd.read_csv("X_train.csv", sep=',')
y_train = pd.read_csv("y_train.csv", sep=',')
y_train = y_train.values.ravel()  # Converting y_train to a 1D array

# Convert y_train to a DataFrame
y_train_df = pd.DataFrame({'ack': y_train})  # Replace 'target_column_name' with the actual column name in y_train

merged_train_data = pd.concat([X_train, y_train_df], axis=1)
train_data = merged_train_data

X_test = pd.read_csv("X_test.csv", sep=',')
y_test = pd.read_csv("y_test.csv", sep=',')

merged_test_data = pd.concat([X_test, y_test], axis=1)
test_data = merged_test_data


One Rule algoritmus sme naprogramovali iba pre číselné hodnoty, pretože v dátach sme si v minulých fázach premenili pomocou encodingu všetky kategorické hodnoty na číselné.

In [ ]:
def one_rule_algorithm(data, target_variable, explored_columns):
    best_feature = None
    best_accuracy = 0
    
    for feature in data.columns:
        if feature == target_variable or feature in explored_columns:
            continue
        
        mean_values = data.groupby(target_variable)[feature].mean()
        split_point = mean_values.mean()
        
        temp_data = data.copy()
        temp_data['prediction'] = temp_data[feature] > split_point
        
        accuracy = accuracy_score(temp_data[target_variable], temp_data['prediction'])
        precision = precision_score(temp_data[target_variable], temp_data['prediction'])
        recall = recall_score(temp_data[target_variable], temp_data['prediction'])
        
        if accuracy > best_accuracy:
            best_feature = feature
            best_accuracy = accuracy
            best_precision = precision
            best_recall = recall
            result = temp_data['prediction']
    
    return best_feature, best_accuracy, best_precision, best_recall, result

Spustíme algoritmus OneR s predikovanou premennou "ack" a vyhodnotíme metriky.

Na vyhodnotenie modelu sme použili nasledovné metriky:
- Accuracy - ako často klasiﬁkátor správne klasifikoval 

- Precision - koľko správne predikovaných príkladov bolo pozitívnych

- Recall - koľko pozitívnych príkladov bolo správne predikovaných

### OneR s jednou premennou

In [ ]:
target_variable = 'ack'

oneR_train_data = copy.deepcopy(train_data)
oneR_test_data = copy.deepcopy(test_data)

best_feature, best_accuracy, best_precision, best_recall, prediction = one_rule_algorithm(oneR_train_data, target_variable, [])
print('Best column from OneR:', best_feature)
print('Accuracy:', best_accuracy)
print('\n')

oneR_train_data[[best_feature, 'ack']]
best_feature, best_accuracy, best_precision, best_recall, prediction = one_rule_algorithm(oneR_test_data, target_variable, [])
print('Best column from OneR:', best_feature)
print('Accuracy:', best_accuracy)
print('Precision:', best_precision)
print('Recall:', best_recall)


Vidíme, že algoritmus vyhodnotil "page_activity_duration" ako najlepšiu premennú pre predikciu "ack". Algoritmus na základe premennej "page_activity_duration" predikoval s presnosťou takmer 90%.

In [ ]:
plt.figure(figsize=(10, 5))
sns.kdeplot(train_data[train_data['ack'] == 0][best_feature], label='ack = 0')
sns.kdeplot(train_data[train_data['ack'] == 1][best_feature], label='ack = 1')
plt.legend()
plt.show()


### OneR s viacerými premennými

Teraz skúsime spustiť OneR algoritmus viackrát a zistiť ktoré premenné najlepšie predikujú "ack".

Po zistení najlepších premenných ich použijeme na vytvorenie modelu. Pri vytváraní modelu prihliadame na presnosť predikcie danej premennej. Čím vyššia presnosť, tým má vyššiu váhu pri rozhodaovaní. 

In [ ]:
target_variable = 'ack'
results = []
explored_columns = []
final_results = []

for _ in range(5):
    oneR_train_data = copy.deepcopy(train_data)
    best_feature, best_accuracy, best_precision, best_recall, prediction = one_rule_algorithm(oneR_train_data, target_variable, explored_columns)
    explored_columns.append(best_feature)
    results.append((best_accuracy, prediction.tolist()))

for result in results:
    if result[0] > 0.7:
        final_results.append(result[1])
        final_results.append(result[1])
    else:
        final_results.append(result[1])

combined_predictions = [Counter(sample).most_common(1)[0][0] for sample in zip(*final_results)]

overall_accuracy = accuracy_score(train_data[target_variable], combined_predictions)
overall_precision = precision_score(train_data[target_variable], combined_predictions, zero_division=0)
overall_recall = recall_score(train_data[target_variable], combined_predictions)

table_data = {
    "Algorithm": ["OneR"],
    "Accuracy": [overall_accuracy],
    "Precision": [overall_precision],
    "Recall": [overall_recall]
}

oneR_table = pd.DataFrame(table_data)

Najlepšie premenné k predikcií "ack"

In [ ]:
table_data = {
    "Feature": explored_columns,
    "Accuracy": [result[0] for result in results]
}

accuracy_table = pd.DataFrame(table_data)
accuracy_table

Výsledky metrík pre predikovanie premennej "ack" pomocou OneR algoritmu s viacerými premennými.

In [ ]:
print('Overall Accuracy:', overall_accuracy)
print('Overall Precision:', overall_precision)
print('Overall Recall:', overall_recall)

### Scikit-learn Klasifikátory

Stromový klasifikátor Random Forest.

In [ ]:
"""# Analysis for varying model complexity
results = []
max_depth_range = range(1, 15)

for depth in max_depth_range:
    rf_model = RandomForestClassifier(n_estimators=100, max_depth=depth, random_state=1)
    rf_model.fit(X_train, y_train)
    
    # Calculate error on training set
    train_pred = rf_model.predict(X_train)
    train_error = 1 - accuracy_score(y_train, train_pred)

    # Calculate error on test set
    test_pred = rf_model.predict(X_test)
    test_error = 1 - accuracy_score(y_test, test_pred)

    results.append({'max_depth': depth, 'train_error': train_error, 'test_error': test_error})

# Convert results to a DataFrame for easier plotting
results_df = pd.DataFrame(results)

# Find the max_depth with the lowest test error
optimal_depth = results_df['max_depth'][results_df['test_error'].idxmin()]

# Plotting the errors vs model complexity
plt.figure(figsize=(10, 6))
plt.plot(results_df['max_depth'], results_df['train_error'], label='Training Error', marker='o', color='blue')
plt.plot(results_df['max_depth'], results_df['test_error'], label='Testing Error', marker='o', color='red')

# Add vertical line at the point of lowest test error
plt.axvline(x=optimal_depth, color='green', linestyle='--', label=f'Optimal Depth: {optimal_depth}')

plt.title("Error vs. Model Complexity (Max Depth)")
plt.xlabel("Max Depth")
plt.ylabel("Error (1 - Accuracy)")
plt.legend()
plt.grid()
plt.show()

# Display the optimal depth and corresponding errors
optimal_info = results_df.loc[results_df['max_depth'] == optimal_depth]
optimal_info"""

In [ ]:
results = []
estimators = []

clf = RandomForestClassifier(max_depth = 17, min_samples_split = 6, min_samples_leaf = 4)
clf.fit(X_train, y_train)

predictions = clf.predict(X_test)
train_predictions = clf.predict(X_train)

accuracy = accuracy_score(y_test, predictions)
accuracy_train = accuracy_score(y_train, train_predictions)
precision = precision_score(y_test, predictions)
recall = recall_score(y_test, predictions)

print(f"Accuracy Train: {accuracy_train:.2f}")
print('\n')
print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")

table_data = {
    "Algorithm": ["Random Forest"],
    "Accuracy": [accuracy],
    "Precision": [precision],
    "Recall": [recall]
}

RF_table = pd.DataFrame(table_data)

feature_importances = clf.feature_importances_
sorted_indices = np.argsort(feature_importances)[::-1]
top_5_features_rf = sorted_indices[:5]
top_5_importances = feature_importances[top_5_features_rf]

feature_names = X_test.columns.values

top_5_feature_names_rf = [feature_names[i] for i in top_5_features_rf]

Stromový klasifikátor Gradient Boosting.

In [ ]:
"""# Analysis for varying model complexity
results = []
max_depth_range = range(1, 15) 

for depth in max_depth_range:
    gb_model = GradientBoostingClassifier(max_depth=depth)
    gb_model.fit(X_train, y_train)
    
    # Calculate error on training set
    train_pred = gb_model.predict(X_train)
    train_error = 1 - accuracy_score(y_train, train_pred)

    # Calculate error on test set
    test_pred = gb_model.predict(X_test)
    test_error = 1 - accuracy_score(y_test, test_pred)

    results.append({'max_depth': depth, 'train_error': train_error, 'test_error': test_error})

# Convert results to a DataFrame for easier plotting
results_df = pd.DataFrame(results)

# Find the max_depth with the lowest test error
optimal_depth = results_df['max_depth'][results_df['test_error'].idxmin()]

# Plotting the errors vs model complexity
plt.figure(figsize=(10, 6))
plt.plot(results_df['max_depth'], results_df['train_error'], label='Training Error', marker='o', color='blue')
plt.plot(results_df['max_depth'], results_df['test_error'], label='Testing Error', marker='o', color='red')

# Add vertical line at the point of lowest test error
plt.axvline(x=optimal_depth, color='green', linestyle='--', label=f'Optimal Depth: {optimal_depth}')

plt.title("Error vs. Model Complexity (Max Depth)")
plt.xlabel("Max Depth")
plt.ylabel("Error (1 - Accuracy)")
plt.legend()
plt.grid()
plt.show()

# Display the optimal depth and corresponding errors
optimal_info = results_df.loc[results_df['max_depth'] == optimal_depth]
optimal_info"""

In [ ]:
gb_model = GradientBoostingClassifier(n_estimators=100, random_state=1, max_depth = 3)

gb_model.fit(X_train, y_train)

gb_y_pred = gb_model.predict(X_test)
gb_y_pred_train = gb_model.predict(X_train)

gb_trin_accuracy = accuracy_score(y_train, gb_y_pred_train)
gb_accuracy = accuracy_score(y_test, gb_y_pred)
gb_precision = precision_score(y_test, gb_y_pred)
gb_recall = recall_score(y_test, gb_y_pred)

print(f"Accuracy Train: {gb_trin_accuracy:.2f}")
print('\n')
print(f"Accuracy: {gb_accuracy:.2f}")
print(f"Precision: {gb_precision:.2f}")
print(f"Recall: {gb_recall:.2f}")

table_data = {
    "Algorithm": ["Gradient Boosting"],
    "Accuracy": [gb_accuracy],
    "Precision": [gb_precision],
    "Recall": [gb_recall]
}

GB_table = pd.DataFrame(table_data)

feature_importances = gb_model.feature_importances_
sorted_indices = np.argsort(feature_importances)[::-1]
top_5_features_gb = sorted_indices[:5]
top_5_importances = feature_importances[top_5_features_gb]
feature_names = X_test.columns.values

top_5_feature_names = [feature_names[i] for i in top_5_features_gb]

Max depth by nám stačilo nastaviť na hĺbku 3, pretože pri vyšších hĺbkach sa model začal pretrénovať.

### Nestromový algoritmus

Pre nestromový algoritmus sme si vybrali KNN klasifikátor.

Najskôr si dáta scalujeme a normalizujeme, aby KNN klasifikátor pracoval efektívnejšie.

In [ ]:
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(X_train)
train_data_scaled = pd.DataFrame(scaled_data, columns=X_train.columns)

power = PowerTransformer(method='yeo-johnson', standardize=True) 
X_train_normalised = power.fit_transform(train_data_scaled)
X_train_scaled_normalised = pd.DataFrame(X_train_normalised, columns=X_train.columns)

scaled_test_data = scaler.transform(X_test)
test_data_scaled = pd.DataFrame(scaled_test_data, columns=X_test.columns)

normalized_test_data = power.transform(test_data_scaled)
test_data_scaled_normalized = pd.DataFrame(normalized_test_data, columns=X_test.columns)

Pri stromových klasifikačných algoritmoch sme nemuseli použiť feature selection, pretože si s tým poradili samé. Pri nestromových algoritmoch je to však inak.

In [ ]:
columns_to_keep = [
    'page_activity_duration', 'pct_doubleclick', 'pct_mouse_click', 'wild_mouse_duration',
    'pct_mouse_move', 'pct_input', 'pct_click', 'session_start', 
    'scroll_move_total_rel_distance', 'pct_scroll_move'
]

X_train_filtered = X_train_scaled_normalised[columns_to_keep]
X_test_filtered = test_data_scaled_normalized[columns_to_keep]

KNN klasifikátor

In [ ]:
knn_model = KNeighborsClassifier(n_neighbors=10)

knn_model.fit(X_train_filtered, y_train)

knn_y_pred = knn_model.predict(X_test_filtered)
knn_y_pred_train = knn_model.predict(X_train_filtered)

knn_trin_accuracy = accuracy_score(y_train, knn_y_pred_train)
knn_accuracy = accuracy_score(y_test, knn_y_pred)
knn_precision = precision_score(y_test, knn_y_pred)
knn_recall = recall_score(y_test, knn_y_pred)

print(f"Accuracy Train: {knn_trin_accuracy:.2f}")
print('\n')
print(f"Accuracy: {knn_accuracy:.2f}")
print(f"Precision: {knn_precision:.2f}")
print(f"Recall: {knn_recall:.2f}")

table_data_knn = {
    "Algorithm": ["K-Nearest Neighbors"],
    "Accuracy": [knn_accuracy],
    "Precision": [knn_precision],
    "Recall": [knn_recall]
}

knn_table = pd.DataFrame(table_data_knn)


selector = SelectKBest(score_func=f_classif, k=5)
selector.fit(X_train_filtered, y_train)
top_5_indices = selector.get_support(indices=True)
top_5_features_knn = X_train_filtered.columns[top_5_indices]

In [ ]:
"""results = []
neighbours = range(1, 15)

for neighbour in neighbours:
    knn_model = KNeighborsClassifier(n_neighbors=neighbour)
    knn_model.fit(X_train_filtered, y_train)
    
    # Calculate error on training set
    train_pred = knn_model.predict(X_train_filtered)
    train_error = 1 - accuracy_score(y_train, train_pred)

    # Calculate error on test set
    test_pred = knn_model.predict(X_test_filtered)
    test_error = 1 - accuracy_score(y_test, test_pred)

    results.append({'n_neighbors': neighbour, 'train_error': train_error, 'test_error': test_error})

# Convert results to a DataFrame for easier plotting
results_df = pd.DataFrame(results)

# Find the number of neighbors with the lowest test error
optimal_neighbors = results_df['n_neighbors'][results_df['test_error'].idxmin()]

# Plotting the errors vs model complexity
plt.figure(figsize=(10, 6))
plt.plot(results_df['n_neighbors'], results_df['train_error'], label='Training Error', marker='o')
plt.plot(results_df['n_neighbors'], results_df['test_error'], label='Testing Error', marker='o')

# Add vertical line at the point of lowest test error
plt.axvline(x=optimal_neighbors, color='green', linestyle='--', label=f'Optimal Neighbors: {optimal_neighbors}')

plt.title("Error vs. Model Complexity (Number of Neighbors)")
plt.xlabel("Number of Neighbors")
plt.ylabel("Error (1 - Accuracy)")
plt.legend()
plt.grid()
plt.show()

# Display the optimal number of neighbors and corresponding errors
optimal_info = results_df.loc[results_df['n_neighbors'] == optimal_neighbors]
print(optimal_info)"""

#### Porovnanie výsledkov

In [ ]:
display(oneR_table, RF_table, GB_table, knn_table)

Z porovnania výsledkov vidíme, že najlepšie výsledky dosiahol Gradient Boosting klasifikátor.

Náš oneR algoritmus dosiahol presnosť 83%. Dôvodom je jednoduchosť algoritmu a potreba predikovať pomocou viacerých premenných. Keďže najlepší atribút má presnosť okolo 90% a ostané atribúty majú presnosť len okolo 60%, tak pri predikovaní pomocou viacerých atribútov sa presnosť zníži.

OneR algoritmus predikuje s precíznosťou 82%, čo je o viac ako 10% menej ako Gradient Boosting klasifikátor alebo Random Forest klasifikátor.

Náš algoritmus je o 15% horší ako ako Gradient Boosting klasifikátor alebo Random Forest klasifikátor v predikovaní pozitívnych príkladov zo všetkých pozitívnych príkladov.

In [ ]:
print("One Rule Algorithm:")
for column in explored_columns:
    print(column)


print("\n")
print("Random Forest:")
for column in top_5_feature_names_rf:
    print(column)


print("\n")
print("Gradient Boosting:")
for column in top_5_feature_names:
    print(column)

print("\n")
print("K Nearest Neighbours:")
for column in top_5_features_knn:
    print(column)

Vidíme, že algoritmy sa podobajú vo výbere atribútov, ktoré najlepšie predikujú "ack". Najlepšie atribúty sú "page_activity_duration", "pct_doubleclick" a "pct_click".

#### Vizualizácia natrénovaných pravidiel

Vizualizujeme si Random Forest klasifikátor.

In [ ]:
tree_index = 0
plt.figure(figsize=(12, 8))
plot_tree(clf.estimators_[tree_index], filled=True, feature_names=[f"feature_{i}" for i in range(X_test.shape[1])])
plt.title(f"Decision Tree {tree_index + 1} from Random Forest Classifier")
plt.show()

Vizualizácia pravidiel pre Random Forest klasifikátor na základe accuracy.

In [ ]:
"""feature_accuracies = []

for i in range(X_train.shape[1]):
    X_train_single_feature = X_train.iloc[:, i:i+1]  
    X_test_single_feature = X_test.iloc[:, i:i+1]   
    
    clf_single_feature = RandomForestClassifier()
    clf_single_feature.fit(X_train_single_feature, y_train)  
    
    predictions_single_feature = clf_single_feature.predict(X_test_single_feature)
    
    accuracy_single_feature = accuracy_score(y_test, predictions_single_feature)
    feature_accuracies.append((X_train.columns[i], accuracy_single_feature))

sorted_features = sorted(feature_accuracies, key=lambda x: x[1], reverse=True)

top_5_feature_names = [feat[0] for feat in sorted_features[:5]]
top_5_accuracies = [feat[1] for feat in sorted_features[:5]]

plt.figure(figsize=(10, 6))
plt.bar(top_5_feature_names, top_5_accuracies, color='skyblue')
plt.xlabel('Feature Name')
plt.ylabel('Accuracy')
plt.title('Top 5 Features with Accuracies')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()"""

In [ ]:
# drop other columns
X_train_filtered = X_train[top_5_feature_names]
X_test_filtered = X_test[top_5_feature_names]

### Hyperparameter tuning

#### Gradient Boosting classifier

Gradient Boosting je veľmi presný model, obzvlášť účinný pri práci s komplexnými dátami, čo je výhodné v rôznych aplikáciách od klasifikácie po regresiu. Jeho flexibilita umožňuje prispôsobenie rozličným typom stratových funkcií, čo ho robí vhodným pre rôzne prediktívne úlohy. Okrem toho, ponúka viacero možností na kontrolu overfittingu, ako napríklad nastavenie hĺbky stromu a rýchlosti učenia, čo zlepšuje jeho schopnosť generalizovať na nevidené dáta.

##### Vysvetlenie hyperparametrov

In [ ]:
params = GradientBoostingClassifier().get_params()
for key, value in params.items():
    print(f"{key}: {value}")

- ccp_alpha: Komplexita stromu. Čím vyššia hodnota, tým viac sa strom zjednoduší.
- criterion: Kritérium pre výber atribútu, ktorý sa použije na rozdelenie uzla.
- init: Inicializácia stromu.
- learning_rate: Rýchlosť učenia.
- loss: Funkcia straty.
- max_depth: Maximálna hĺbka stromu.
- max_features: Maximálny počet atribútov, ktoré sa použijú na rozdelenie uzla.
- max_leaf_nodes: Maximálny počet listových uzlov.
- min_impurity_decrease: Uzol sa rozdelí, ak sa impurity v jeho rodičovskom uzle zníži o túto hodnotu.
- min_samples_leaf: Minimálny počet vzoriek v listovom uzle.
- min_weight_fraction_leaf: Minimálna váhová frakcia v listovom uzle.
- n_estimators: Počet stromov v ensemble.
- n_iter_no_change: Počet iterácií bez zlepšenia pred ukončením učenia.
- random_state: Seed pre generovanie náhodných čísel.
- subsample: Podiel vzoriek použitých na trénovanie každého stromu.
- tol: Tolerancia pre zastavenie učenia.
- validation_fraction: Podiel vzoriek použitých na validáciu každého stromu.
- verbose: Výpis informácií o učení.
- warm_start: Použiť existujúci model na trénovanie a pridať ďalšie stromy do ensemble.

In [ ]:
# Adjusted parameter grid to combat overfitting
gb_parameters = {
    'criterion': ['friedman_mse', 'squared_error'],
    'loss': ['log_loss', 'exponential'],
    'max_depth': [1, 2, 3, 4],
    'n_estimators': [80, 90, 100, 110, 120],
    'learning_rate': [0.09, 0.1, 0.11],
    'subsample': [0.8, 0.9, 1.0],  # Adding subsample parameter
    'min_samples_split': [3, 4, 5],
    'min_samples_leaf': [2, 3, 4],
    'random_state': [None, 1]
}

# Initialize RandomizedSearchCV
gb_clf = RandomizedSearchCV(
    estimator=GradientBoostingClassifier(),
    param_distributions=gb_parameters,
    cv=5,
    n_iter=10,
    n_jobs=-1
)

# Fit the random search
gb_search = gb_clf.fit(X_train, y_train)

# Output the results
print("Best Estimator:", gb_search.best_estimator_)
print("Best Score:", gb_search.best_score_)
print("Best Parameters:", gb_search.best_params_)
print("Number of splits:", gb_search.n_splits_)

# Train the best model with early stopping
best_gb_model = gb_search.best_estimator_
best_gb_model.n_iter_no_change = 5
best_gb_model.validation_fraction = 0.1
best_gb_model.fit(X_train, y_train)

# Feature importances and reduction
feature_importances = best_gb_model.feature_importances_
N = 10  # Top N features
top_features_indices = feature_importances.argsort()[-N:][::-1]
top_features = [X_train.columns[i] for i in top_features_indices]
X_train_reduced = X_train[top_features]
X_test_reduced = X_test[top_features]

# Train and evaluate on the reduced feature set
final_model = best_gb_model.fit(X_train_reduced, y_train)
final_model_score = final_model.score(X_test_reduced, y_test)
print("Final Model Score on Test Data:", final_model_score)


##### Gradient Boosting before hyperparameter tuning

In [ ]:
gb_model = GradientBoostingClassifier()

gb_model.fit(X_train, y_train)

gb_y_pred = gb_model.predict(X_test)

gb_accuracy = accuracy_score(y_test, gb_y_pred)
gb_precision2 = precision_score(y_test, gb_y_pred)
gb_recall2 = recall_score(y_test, gb_y_pred)

print(f"Accuracy: {gb_accuracy:.5f}")
print(f"Precision: {gb_precision2:.5f}")
print(f"Recall: {gb_recall2:.5f}")

In [ ]:
def evaluate_model_performance(clf, X_train, y_train, X_test, y_test):
    # Fit the model
    clf.fit(X_train, y_train)

    # Predictions on training and test data
    train_predictions = clf.predict(X_train)
    test_predictions = clf.predict(X_test)

    # Calculate metrics for training data
    train_accuracy = accuracy_score(y_train, train_predictions)
    train_precision = precision_score(y_train, train_predictions)
    train_recall = recall_score(y_train, train_predictions)

    # Calculate metrics for test data
    test_accuracy = accuracy_score(y_test, test_predictions)
    test_precision = precision_score(y_test, test_predictions)
    test_recall = recall_score(y_test, test_predictions)

    # Print training metrics
    print("Training Metrics:")
    print(f"Accuracy: {train_accuracy:.5f}")
    print(f"Precision: {train_precision:.5f}")
    print(f"Recall: {train_recall:.5f}\n")

    # Print test metrics
    print("Test Metrics:")
    print(f"Accuracy: {test_accuracy:.5f}")
    print(f"Precision: {test_precision:.5f}")
    print(f"Recall: {test_recall:.5f}\n")

    # Check for Underfitting
    if train_accuracy < 0.6 and test_accuracy < 0.6:
        print("The model is underfitting.")
    else:
        print(f"The model is not underfitting because the training and test accuracies are {train_accuracy:.5f} and {test_accuracy:.5f}.")

    # Check for Overfitting
    if train_accuracy > 0.8 and (train_accuracy - test_accuracy) > 0.1 or train_accuracy == 1.0:
        print("The model is overfitting.")
    else:
        print(f"The model is not overfitting because the difference between the training and test accuracies is {train_accuracy - test_accuracy:.5f}.")

    # Check for Stability
    if abs(train_accuracy - test_accuracy) < 0.1:
        print(f"The model is stable because the difference between the training and test accuracies is {abs(train_accuracy - test_accuracy):.5f}.")
    else:
        print("The model is not stable.")


##### Gradient Boosting after hyperparameter tuning

In [ ]:
clf = gb_search.best_estimator_
evaluate_model_performance(clf, X_train_reduced, y_train, X_test_reduced, y_test)

#### Random Forest classifier

Random Forest je obľúbený pre svoju robustnosť a výkonnosť, ktorá je často porovnateľná s komplexnejšími modelmi. Vďaka svojmu spôsobu náhodnej výberu prvkov a tvorby stromov poskytuje Random Forest vynikajúcu odolnosť voči overfittingu. Táto metóda je tiež schopná efektívne spracovávať veľké dátove sady s veľkým počtom premenných, čo je výhodné pri riešení širokej škály úloh prediktívnej analýzy.

##### Vysvetlenie hyperparametrov

In [ ]:
params = RandomForestClassifier().get_params()
for key, value in params.items():
    print(f"{key}: {value}")

- bootstrap: Použiť bootstrap vzorkovanie. Bootstrap vzorkovanie je metóda, ktorá sa používa na zlepšenie presnosti modelu.
- class_weight: Váhy tried.
- max_samples: Maximálny počet vzoriek použitých na trénovanie každého stromu.
-  n_jobs: Počet jadier procesora použitých na trénovanie.
-  oob_score: Použiť out-of-bag vzorky na odhad presnosti. Out-of-bag vzorky sú vzorky, ktoré neboli použité na trénovanie stromu.

In [ ]:
"""estimator = RandomForestClassifier()

# Random Forest Parameters
rf_parameters = {
    'n_estimators': [80,90,100,110,120],
    'max_depth': [ 3,4,5,6,7],
    'min_samples_split': [2, 3,4],
    'min_samples_leaf': [1, 2,3,4],
    'bootstrap': [True, False],
    'criterion': ['gini', 'entropy'],
    'max_features': ['sqrt', 'log2']
}

scoring = ['accuracy', 'precision_micro']

rf_clf = RandomizedSearchCV(
    estimator=estimator, 
    param_distributions=rf_parameters, 
    cv=5, 
    n_jobs=-1,
    n_iter=10,
)

rf_search = rf_clf.fit(X_train, y_train)

# Output the results
print("Best Estimator:", rf_search.best_estimator_)
print("Best Score:", rf_search.best_score_)
print("Best Parameters:", rf_search.best_params_)
print("Number of Splits:", rf_search.n_splits_)

# Feature Importance and Selection
best_rf_model = rf_search.best_estimator_.fit(X_train, y_train)
rf_feature_importances = best_rf_model.feature_importances_

# Select the top N features (e.g., top 5)
N = 5
rf_top_features_indices = rf_feature_importances.argsort()[-N:][::-1]
rf_top_features = [X_train.columns[i] for i in rf_top_features_indices]

# Restrict your training and testing sets to these top features
X_train_reduced = X_train[rf_top_features]
X_test_reduced = X_test[rf_top_features]"""

##### Random Forest before hyperparameter tuning

In [ ]:
"""clf = RandomForestClassifier()
clf.fit(X_train, y_train)

predictions = clf.predict(X_test)

accuracy = accuracy_score(y_test, predictions)
precision = precision_score(y_test, predictions)
recall = recall_score(y_test, predictions)

print(f"Accuracy: {accuracy:.5f}")
print(f"Precision: {precision:.5f}")
print(f"Recall: {recall:.5f}")"""

##### Random Forest after hyperparameter tuning

In [ ]:
"""clf = rf_search.best_estimator_
evaluate_model_performance(clf, X_train_reduced, y_train, X_test_reduced, y_test)"""

#### KNeighbors classifier

K-Nearest Neighbors (KNN) je známy svojou jednoduchosťou a intuitívnosťou, čo uľahčuje jeho pochopenie a implementáciu. Jeho flexibilita výberu počtu susedov a metrík vzdialenosti umožňuje prispôsobiť model špecifickým potrebám dát. Navyše, KNN je efektívny pri zachytávaní komplexných vzorov v dátach, čo je výhodné, keď sú presné predpovede založené na podobnosti dát kľúčové.

##### Vysvetlenie hyperparametrov


In [ ]:
params = KNeighborsClassifier().get_params()
for key, value in params.items():
    print(f"{key}: {value}")

- algorithm: Algoritmus použitý na nájdenie najbližších susedov. Algoritmus môže byť "ball_tree", "kd_tree", "brute" alebo "auto".
- leaf_size: Veľkosť listového uzla.
- metric: Metrika použitá na výpočet vzdialenosti medzi bodmi. Metrika môže byť "euclidean", "manhattan" alebo "minkowski".
- n_neighbors: Počet najbližších susedov.
- p: Parameter pre metriku "minkowski".
- weights: Váhy použité na predikciu bodu. Váhy môžu byť "uniform" alebo "distance".

In [ ]:
"""# KNN Parameters
knn_parameters = {
    'n_neighbors': [5, 6, 7, 8, 9, 10],
    'weights': ['uniform', 'distance'],
    'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
    'leaf_size': [20, 25, 30, 35, 40],
    'metric': ['euclidean', 'manhattan', 'minkowski'],
    'p': [1, 2]
}

scoring = ['accuracy', 'precision_micro']

knn_clf = RandomizedSearchCV(
    estimator=KNeighborsClassifier(),
    param_distributions=knn_parameters,
    scoring=scoring,
    refit='accuracy',
    cv=5,
    n_jobs=-1
)

knn_search = knn_clf.fit(X_train_reduced, y_train)

# Output the results
print("Best Estimator:", knn_search.best_estimator_)
print("Best Score:", knn_search.best_score_)
print("Best Parameters:", knn_search.best_params_)
print("Number of Splits:", knn_search.n_splits_)"""

##### KNN before hyperparameter tuning

In [ ]:
"""knn_model = KNeighborsClassifier()

knn_model.fit(X_train_filtered, y_train)

knn_y_pred = knn_model.predict(X_test_filtered)

knn_accuracy = accuracy_score(y_test, knn_y_pred)
knn_precision = precision_score(y_test, knn_y_pred)
knn_recall = recall_score(y_test, knn_y_pred)

print(f"Accuracy: {knn_accuracy:.5f}")
print(f"Precision: {knn_precision:.5f}")
print(f"Recall: {knn_recall:.5f}")"""

##### KNN after hyperparameter tuning

In [ ]:
"""clf = knn_search.best_estimator_
evaluate_model_performance(clf, X_train_reduced, y_train, X_test_reduced, y_test)"""

#### Voting ensemble 

In [ ]:
"""# Create individual models with their best parameters
model1 = gb_search.best_estimator_
model2 = rf_search.best_estimator_
model3 = knn_search.best_estimator_

# Create a voting classifier
voting_clf = VotingClassifier(
    estimators=[('gb', model1), ('rf', model2), ('knn', model3)],
    voting='hard'
)

# Fit the voting classifier
voting_clf.fit(X_train, y_train)

# Evaluate the model
voting_score = voting_clf.score(X_test, y_test)
print(f"Voting Classifier Score: {voting_score}")
"""

In [ ]:
"""clf = voting_clf
evaluate_model_performance(clf, X_train_reduced, y_train, X_test_reduced, y_test)"""

#### Stacking ensemble

In [ ]:
"""# Create individual models with their best parameters
model1 = gb_search.best_estimator_
model2 = rf_search.best_estimator_
model3 = knn_search.best_estimator_

# Define the stacking ensemble
stacking_clf = StackingClassifier(
    estimators=[('gb', model1), ('rf', model2), ('knn', model3)],
    final_estimator=LogisticRegression(),
    cv=5
)

# Fit the stacking classifier
stacking_clf.fit(X_train, y_train)

# Evaluate the model
stacking_score = stacking_clf.score(X_test, y_test)
print(f"Stacking Classifier Score: {stacking_score}")
"""

In [ ]:
"""clf = stacking_clf
evaluate_model_performance(clf, X_train_reduced, y_train, X_test_reduced, y_test)"""

#### Voľba najlepšieho modelu
Pre nasadenie modelu sme si vybrali Gradient Boosting klasifikátor, pretože dosiahol najlepšie výsledky. Nevykazoval ani overfitting, ani underfitting a bol stabilný. V porovnaní s ostatnými modelmi dosiahol najlepšie výsledky.

### Vyhodnotenie vplyvu zvolenej stratégie riešenia na klasifikáciu

#### Hyperparameter tuning resp. ensemble learning

robilo sa v Laurinej casti

#### Najlepší model pre nasadenie (deployment)? 

#### Data pipeline pre jeho vybudovanie

### Najlepší model

Načítame nespracované dáta

In [ ]:
user = pd.read_csv("user_phase_1.csv", sep='\t')
session = pd.read_csv("session_phase_1.csv", sep='\t')
data= pd.merge(user, session, on='user_id', how='inner')

Rozdelíme si ich na trénovaciu a testovaciu množinu, dávame si pozor na rovnomernú distribúciu.

In [ ]:
from sklearn.model_selection import train_test_split

X = data.drop('ack', axis=1)
y = data['ack']  

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

train_target_distribution = y_train.value_counts(normalize=True)
test_target_distribution = y_test.value_counts(normalize=True)

# Compare distributions
print("Train Target Distribution:")
print(train_target_distribution)
print("\nTest Target Distribution:")
print(test_target_distribution)

In [ ]:
# Function to evaluate the model
def evaluate_model(model, X_train, y_train, X_test, y_test):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    return accuracy, precision, recall

Vytvoríme si funkciu na inputation of missing variables a na encoding kategorických premenných. Vo funkcii taktiež detekujeme outlierov a nahrádzame ich priemerom.

In [ ]:
from copy import deepcopy
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy='mean')
columns_to_drop = ['sex', 'race', 'browser_name']

numeric_columns = [
    'pct_scroll_move', 
    'pct_doubleclick', 
    'pct_input', 
    'page_activity_duration', 
    'pct_click',
    'pct_mouse_move',
    'pct_scrandom', 
    'pct_scroll_move_duration', 
    'mouse_move_total_rel_distance',
    'pct_rage_click',
    'pct_wild_mouse', 
    'wild_mouse_duration', 
    'pct_click_product_info', 
    'scroll_move_total_rel_distance',
    'pct_mouse_click'
]

object_columns = [
    'birthdate',
    'registration',
    'session_start'
]


In [ ]:
def missing_and_outliers(X):
    A = X.copy()
    A = A.drop(columns=columns_to_drop, axis=1)
    A[numeric_columns] = imputer.fit_transform(A[numeric_columns])
    A.dropna(inplace=True)
    for col in object_columns:
        A[col] = pd.to_datetime(A[col], errors='coerce')
        A[f'numeric_{col}'] = A[col].dt.strftime('%Y%m%d').astype(float)
        A.drop(col, axis=1, inplace=True)
        A.rename(columns={f'numeric_{col}': col}, inplace=True)

    for col in numeric_columns:
        mean_value = A[col].mean()
        std_value = A[col].std()
        z_scores = (A[col] - mean_value) / std_value
        outlier_indices = np.abs(z_scores) > 3
        A.loc[outlier_indices, col] = mean_value

    return A

def drop_missing_and_outliers(X):
    A = X.copy()
    A = A.drop(columns=columns_to_drop, axis=1)
    A[numeric_columns] = imputer.fit_transform(A[numeric_columns])
    A.dropna(inplace=True)
    for col in object_columns:
        A[col] = pd.to_datetime(A[col], errors='coerce')
        A[f'numeric_{col}'] = A[col].dt.strftime('%Y%m%d').astype(float)
        A.drop(col, axis=1, inplace=True)
        A.rename(columns={f'numeric_{col}': col}, inplace=True)

    # Identify and drop outliers from numeric columns
    for col in numeric_columns:
        mean_value = A[col].mean()
        std_value = A[col].std()
        z_scores = (A[col] - mean_value) / std_value
        outlier_indices = np.abs(z_scores) > 3
        A = A.loc[~outlier_indices]

    return A



# Apply the functions to train and test data
merged_train_data_filled = missing_and_outliers(pd.concat([X_train, y_train], axis=1))
merged_test_data_filled = missing_and_outliers(pd.concat([X_test, y_test], axis=1))

merged_train_data_dropped = drop_missing_and_outliers(pd.concat([X_train, y_train], axis=1))
merged_test_data_dropped = drop_missing_and_outliers(pd.concat([X_test, y_test], axis=1))

# Prepare the datasets
X_train_filled, y_train_filled = merged_train_data_filled.drop('ack', axis=1), merged_train_data_filled['ack']
X_test_filled, y_test_filled = merged_test_data_filled.drop('ack', axis=1), merged_test_data_filled['ack']

X_train_dropped, y_train_dropped = merged_train_data_dropped.drop('ack', axis=1), merged_train_data_dropped['ack']
X_test_dropped, y_test_dropped = merged_test_data_dropped.drop('ack', axis=1), merged_test_data_dropped['ack']

# Initialize and evaluate the model
gb_model = GradientBoostingClassifier()

gb_filled_accuracy, gb_filled_precision, gb_filled_recall = evaluate_model(gb_model, X_train_filled, y_train_filled, X_test_filled, y_test_filled)
gb_dropped_accuracy, gb_dropped_precision, gb_dropped_recall = evaluate_model(gb_model, X_train_dropped, y_train_dropped, X_test_dropped, y_test_dropped)


print("Model with filled missing values and filled outliers:")
print(f"Accuracy: {gb_filled_accuracy:.5f}, Precision: {gb_filled_precision:.5f}, Recall: {gb_filled_recall:.5f}")

print("\nModel with filled missing values and dropped outliers:")
print(f"Accuracy: {gb_dropped_accuracy:.5f}, Precision: {gb_dropped_precision:.5f}, Recall: {gb_dropped_recall:.5f}")

# Determine the best model based on accuracy, precision, and recall
best_approach = "Filled missing values and filled outliers" if gb_filled_accuracy >= gb_dropped_accuracy else "Filled missing values and dropped outliers"
print(f"\nBest Model Approach: {best_approach} - Based on Accuracy")

X_train = X_train_filled if gb_filled_accuracy >= gb_dropped_accuracy else X_train_dropped
X_test = X_test_filled if gb_filled_accuracy >= gb_dropped_accuracy else X_test_dropped
y_train = y_train_filled if gb_filled_accuracy >= gb_dropped_accuracy else y_train_dropped
y_test = y_test_filled if gb_filled_accuracy >= gb_dropped_accuracy else y_test_dropped


#### Vyhodnotenie fillingu missing values a fillingu/dropnutiu outlierov
Vyplnenie missing values sme museli vykonať, pretože niektoré algoritmy strojového učenia nepracujú s nimi. Výslendky sme porovnali s výsledkami, ktoré sme dostali pri dropnutí vs fillingu outlierov. Výsledky boli podobné, ale pri fillingu sme dosiahli lepšie výsledky. Preto sme sa rozhodli použiť fillnutie outlierov.

In [ ]:
from sklearn.preprocessing import MinMaxScaler, PowerTransformer

def scale_data(X):
    scaler = MinMaxScaler()
    X_scaled = X.copy()
    X_scaled[numeric_columns] = scaler.fit_transform(X_scaled[numeric_columns])
    return X_scaled

def transform_data(X):
    transformer = PowerTransformer()
    X_transformed = X.copy()
    X_transformed[numeric_columns] = transformer.fit_transform(X_transformed[numeric_columns])
    return X_transformed

def scale_and_transform_data(X):
    scaler = MinMaxScaler()
    transformer = PowerTransformer()
    X_scaled_transformed = X.copy()
    X_scaled_transformed[numeric_columns] = scaler.fit_transform(X_scaled_transformed[numeric_columns])
    X_scaled_transformed[numeric_columns] = transformer.fit_transform(X_scaled_transformed[numeric_columns])
    return X_scaled_transformed

def transform_and_scale_data(X):
    transformer = PowerTransformer()
    scaler = MinMaxScaler()
    X_transformed_scaled = X.copy()
    X_transformed_scaled[numeric_columns] = transformer.fit_transform(X_transformed_scaled[numeric_columns])
    X_transformed_scaled[numeric_columns] = scaler.fit_transform(X_transformed_scaled[numeric_columns])
    return X_transformed_scaled

# Apply the scaling and transforming functions to your data
X_train_scaled = scale_data(X_train)
X_test_scaled = scale_data(X_test)

X_train_transformed = transform_data(X_train)
X_test_transformed = transform_data(X_test)

# Apply the combined scaling and transforming function to your data
X_train_scaled_transformed = scale_and_transform_data(X_train)
X_test_scaled_transformed = scale_and_transform_data(X_test)

X_train_transformed_scaled = transform_and_scale_data(X_train)
X_test_transformed_scaled = transform_and_scale_data(X_test)

# Evaluate model on original data
accuracy_original, precision_original, recall_original = evaluate_model(deepcopy(gb_model), X_train, y_train, X_test, y_test)
print(f"Original Data - Accuracy: {accuracy_original:.5f}, Precision: {precision_original:.5f}, Recall: {recall_original:.5f}")

# Evaluate model on scaled data
accuracy_scaled, precision_scaled, recall_scaled = evaluate_model(deepcopy(gb_model), X_train_scaled, y_train, X_test_scaled, y_test)
print(f"Scaled Data - Accuracy: {accuracy_scaled:.5f}, Precision: {precision_scaled:.5f}, Recall: {recall_scaled:.5f}")

# Evaluate model on transformed data
accuracy_transformed, precision_transformed, recall_transformed = evaluate_model(deepcopy(gb_model), X_train_transformed, y_train, X_test_transformed, y_test)
print(f"Transformed Data - Accuracy: {accuracy_transformed:.5f}, Precision: {precision_transformed:.5f}, Recall: {recall_transformed:.5f}")

# Evaluate model on scaled and transformed data
accuracy_scaled_transformed, precision_scaled_transformed, recall_scaled_transformed = evaluate_model(deepcopy(gb_model), X_train_scaled_transformed, y_train, X_test_scaled_transformed, y_test)
print(f"Scaled & Transformed Data - Accuracy: {accuracy_scaled_transformed:.5f}, Precision: {precision_scaled_transformed:.5f}, Recall: {recall_scaled_transformed:.5f}")

accuracy_transformed_scaled, precision_transformed_scaled, recall_transformed_scaled = evaluate_model(deepcopy(gb_model), X_train_transformed_scaled, y_train, X_test_transformed_scaled, y_test)
print(f"Transformed & Scaled Data - Accuracy: {accuracy_transformed_scaled:.5f}, Precision: {precision_transformed_scaled:.5f}, Recall: {recall_transformed_scaled:.5f}")

# Store the results
results = {
    "Original Data": (accuracy_original, precision_original, recall_original),
    "Scaled Data": (accuracy_scaled, precision_scaled, recall_scaled),
    "Transformed Data": (accuracy_transformed, precision_transformed, recall_transformed),
    "Scaled & Transformed Data": (accuracy_scaled_transformed, precision_scaled_transformed, recall_scaled_transformed),
    "Transformed & Scaled Data": (accuracy_transformed_scaled, precision_transformed_scaled, recall_transformed_scaled)
}

# Determine the best model
best_model = max(results, key=lambda x: (results[x][0], results[x][1], results[x][2]))  # Prioritizing accuracy, then precision, then recall

# Print the best model and its metrics
best_accuracy, best_precision, best_recall = results[best_model]
print(f"\nBest Model: {best_model}")
print(f"Accuracy: {best_accuracy:.5f}, Precision: {best_precision:.5f}, Recall: {best_recall:.5f}")

# Explanation
if best_model == "Original Data":
    explanation = "This model performs best likely due to the nature of the data, which may not require scaling or transformation for optimal results."
elif best_model == "Scaled Data":
    explanation = "Scaling the data to a uniform range has likely improved model performance by equalizing the influence of all features."
elif best_model == "Transformed Data":
    explanation = "Transforming the data to a more Gaussian distribution has likely improved the model's ability to learn from the data."
elif best_model == "Scaled & Transformed Data":
    explanation = "Scaling and transforming the data have both improved the model's performance, indicating the data benefits from both adjustments."
else:
    explanation = "Transforming and scaling the data in this order has likely improved the model's performance, indicating the data benefits from both adjustments."

print(f"Reason: {explanation}")



#### Vyhodnotenie scalingu a transformácie dát

Na našich dátach sme previedli rôzne kombinácie scalingu a transformácie. Použili sme MinMaxScaler a PowerTransformer. Výsledky sme porovnali a zistili, že najlepšie a zároveň rovnaké výsledky sme dosiahli pri transformovaní a scalingu. Preto sme sa rozhodli použiť obe metódy.

In [ ]:
from sklearn.feature_selection import RFE, SelectFromModel
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from sklearn.tree import DecisionTreeClassifier


def select_features(method, model, X, y, k):
    if method == 'SelectKBest':
        selector = SelectKBest(f_classif, k=k)
        X_selected = selector.fit_transform(X, y)
    elif method == 'RandomForestImportance':
        forest = RandomForestClassifier()
        forest.fit(X, y)
        selector = SelectFromModel(forest, threshold=-np.inf, max_features=k)
        X_selected = selector.fit_transform(X, y)

    return X_selected


In [ ]:
import time
import warnings
from sklearn.exceptions import ConvergenceWarning
from tabulate import tabulate


warnings.filterwarnings("ignore", category=ConvergenceWarning)

model_results = {}
feature_counts = range(1,4) 
feature_methods = ['SelectKBest', 'RandomForestImportance']

models = {
    'GradientBoosting': GradientBoostingClassifier(),
    'RandomForest': RandomForestClassifier(),
    'KNN': KNeighborsClassifier(),
    'LogisticRegression': LogisticRegression(),
    'DecisionTree': DecisionTreeClassifier(),
}

for name, model in models.items():
    for method in feature_methods:
        for k in feature_counts:
            X_train_selected = select_features(method, model, X_train, y_train, k)
            X_test_selected = select_features(method, model, X_test, y_test, k)

            # Fit the model on the training data
            start_time = time.time()
            model.fit(X_train_selected, y_train)
            end_time = time.time()

            # Predict on both the training and testing data
            train_predictions = model.predict(X_train_selected)
            test_predictions = model.predict(X_test_selected)

            # Calculate accuracy and run-time for training data
            train_accuracy = accuracy_score(y_train, train_predictions)

            # Calculate accuracy and run-time for testing data
            test_accuracy = accuracy_score(y_test, test_predictions)
            run_time = end_time - start_time

            result_key_train = f"{name}_{method}_Top{k}_Train"
            result_key_test = f"{name}_{method}_Top{k}_Test"

            if train_accuracy or test_accuracy > 0.985: # Ignore models with overfitting
                continue
            else:
                model_results[result_key_train] = {'Accuracy_train': train_accuracy, 'Accuracy_test': test_accuracy, 'Run-time': run_time}



for item in model_results.items():
    print(item)


warnings.filterwarnings("default", category=ConvergenceWarning)


#### Vyhodnotenie výberu atribútov a algoritmov strojového učenia

Použitá literatúra:
- https://github.com/thismlguy/analytics_vidhya/tree/master/Articles/Parameter_Tuning_GBM_with_Example
- https://www.analyticsvidhya.com/blog/2016/02/complete-guide-parameter-tuning-gradient-boosting-gbm-python/
